In [1]:
# Скрипт расчета A/B подготовлен командой EXPF специально для лекций по A/B
# Курс по A/B-тестированиям expf.ru/ab_course
# A/B-платформа по подписке expf.ru/sigma

import pandas as pd

import config as cfg
from metric_builder import _load_yaml_preset
from report import build_experiment_report
import time

logger = cfg.logger
start_time = time.time()

# скачайте отдельно https://drive.google.com/file/d/1f-HM6v5HQFrQ8Rn8DmWz9G4NF4uTbo4x/view?usp=share_link
# df = pd.read_parquet(f'data/parquet/df.parquet')

# Мини-версия таблицы с данными по эксперименту, количество строк = 10000
df = pd.read_csv("data/csv/df_sample.csv")
logger.info("Data loaded")

experiment_report = build_experiment_report(
    df=df,
    metric_config=_load_yaml_preset(preset="todo")
)
experiment_report.to_csv(f"experiment_report.csv")

cfg.logger.info(time.time() - start_time)


[2023-11-25 21:00:20] - AB Pipeline - INFO - Data loaded
[2023-11-25 21:00:22] - AB Pipeline - INFO - Average Revenuer Per User Without Cancelllations
[2023-11-25 21:00:23] - AB Pipeline - INFO - Average Order Value
[2023-11-25 21:00:24] - AB Pipeline - INFO - Canceled Orders per Orders
[2023-11-25 21:00:26] - AB Pipeline - ERROR - The samples x and y must have the same length.
[2023-11-25 21:00:26] - AB Pipeline - INFO - Average Revenue per Paying User Without Cancelation
[2023-11-25 21:00:26] - AB Pipeline - INFO - 5.67645001411438


In [2]:
experiment_report

,metric_name,mean_0,mean_1,var_0,var_1,delta,lift,pvalue,statistic
0,Average Revenuer Per User Without Cancelllations,35.02924,26.452356,85678.243854,66817.364501,-8.576885,-0.244849,0.357896,0.919502
0,Average Order Value,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
0,Canceled Orders per Orders,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.520526,0.642535
0,Average Revenue per Paying User Without Cancel...,35.02924,26.452356,85678.243854,66817.364501,-8.576885,-0.244849,None,None


In [4]:
metric_config=_load_yaml_preset(preset="todo")

In [5]:
df = pd.read_csv("data/csv/df_sample.csv")

In [6]:
import pandas as pd
import numpy as np
import abc
import utils
import config as cfg
from itertools import product
from metric_builder import Metric, CalculateMetric
from stattests import TTestFromStats, calculate_statistics, calculate_linearization, ProportionZFromTest, UTestFromTest

In [25]:
n = 0
for metric_params in metric_config:
    print(metric_params)
    n += 1
    if n == 2:
        break

{'name': 'Average Revenuer Per User Without Cancelllations', 'type': 'ratio', 'estimator': 't_test', 'level': 'user_pseudo_id', 'numerator': {'aggregation_field': 'revenue', 'aggregation_function': 'sum'}, 'numerator_conditions': [{'condition_field': 'pg_status', 'comparison_sign': 'not_equal', 'comparison_value': 'CANCELED'}], 'denominator': {'aggregation_field': 'user_pseudo_id', 'aggregation_function': 'count_distinct'}}
{'name': 'Average Order Value', 'type': 'ratio', 'estimator': 't_test', 'level': 'user_pseudo_id', 'numerator': {'aggregation_field': 'order_id', 'aggregation_function': 'count_distinct', 'numerator_conditions': [{'condition_field': 'pg_status', 'comparison_sign': 'not_equal', 'comparison_value': 'CANCELED'}]}, 'denominator': {'aggregation_field': 'order_id', 'aggregation_function': 'count_distinct'}, 'denominator_conditions': [{'condition_field': 'pg_status', 'comparison_sign': 'equal', 'comparison_value': 'REALIZATION'}]}


In [26]:
metric_params

{'name': 'Average Order Value',
 'type': 'ratio',
 'estimator': 't_test',
 'level': 'user_pseudo_id',
 'numerator': {'aggregation_field': 'order_id',
  'aggregation_function': 'count_distinct',
  'numerator_conditions': [{'condition_field': 'pg_status',
    'comparison_sign': 'not_equal',
    'comparison_value': 'CANCELED'}]},
 'denominator': {'aggregation_field': 'order_id',
  'aggregation_function': 'count_distinct'},
 'denominator_conditions': [{'condition_field': 'pg_status',
   'comparison_sign': 'equal',
   'comparison_value': 'REALIZATION'}]}

In [27]:
metric_parsed = Metric(metric_params)

In [28]:
calculated_metric = CalculateMetric(metric_parsed)(df)

In [29]:
df_ = calculate_linearization(calculated_metric)

In [30]:
stats = calculate_statistics(df_, metric_parsed.type)

In [33]:
if metric_parsed.estimator == 't_test':
    print(metric_parsed.estimator)
    test = TTestFromStats()
    criteria_res = test(stats)
elif metric_parsed.estimator  == 'prop_test':
    test = ProportionZFromTest()
    criteria_res = test(stats)
    print(metric_parsed.estimator)
elif metric_parsed.estimator  == 'mann_whitney':
    test = UTestFromTest()
    criteria_res = test(df_)
    print(metric_parsed.estimator)

t_test


In [34]:
import pandas as pd
import numpy as np
import abc
import statsmodels.stats.proportion as smprop
from scipy.stats import ttest_ind_from_stats, ttest_ind, wilcoxon

In [36]:
stats.mean_0

1.0

In [39]:
pvalue

nan

In [40]:
statistic

nan

In [41]:
mean1=stats.mean_0,
std1=np.sqrt(stats.var_0),
nobs1=stats.n_0,
mean2=stats.mean_1,
std2=np.sqrt(stats.var_1),
nobs2=stats.n_1

In [47]:
nobs2

1754

In [38]:
statistic, pvalue = ttest_ind_from_stats(
                mean1=stats.mean_0,
                std1=np.sqrt(stats.var_0),
                nobs1=stats.n_0,
                mean2=stats.mean_1,
                std2=np.sqrt(stats.var_1),
                nobs2=stats.n_1
)

In [23]:
criteria_res.pvalue

0.5205255920924144

In [24]:
metric_parsed.name

'Canceled Orders per Orders'

In [ ]:

class Report:
    def __init__(self, report):
        self.report = report


class BuildMetricReport:
    def __call__(self, calculated_metric, metric_items) -> Report:


        if metric_items.estimator == 't_test':
            df_ = calculate_linearization(calculated_metric)
            test = TTestFromStats()
            stats = calculate_statistics(df_, metric_items.type)
            criteria_res = test(stats)
        elif metric_items.estimator  == 'prop_test':
            stats = calculate_statistics(df, metric_items.type)
            criteria_res = ProportionZFromTest(stats)
        elif metric_items.estimator  == 'mann_whitney':
            criteria_res = UTestFromTest(calculated_metric)


        cfg.logger.info(f"{metric_items.name}")

        report_items = pd.DataFrame({
            "metric_name": metric_items.name,
            "mean_0": stats.mean_0,
            "mean_1": stats.mean_1,
            "var_0": stats.var_0,
            "var_1": stats.var_1,
            "delta": stats.mean_1 - stats.mean_0,
            "lift":  (stats.mean_1 - stats.mean_0) / stats.mean_0,
            "pvalue": criteria_res.pvalue,
            "statistic": criteria_res.statistic
        }, index=[0])

        return Report(report_items)


def build_experiment_report(df, metric_config):
    build_metric_report = BuildMetricReport()
    reports = []

    for metric_params in metric_config:
        metric_parsed = Metric(metric_params)
        calculated_metric = CalculateMetric(metric_parsed)(df)
        metric_report = build_metric_report(calculated_metric, metric_parsed)
        reports.append(metric_report.report)

    return pd.concat(reports)



In [2]:
_unique_variants = df[cfg.VARIANT_COL].unique()

In [3]:
_unique_variants

array([3, 0, 1, 2])

In [17]:
metric_config=_load_yaml_preset(preset="todo")

In [46]:
metric_params

{'name': 'Canceled Orders per Orders',
 'type': 'ratio',
 'estimator': 'prop_test',
 'level': 'user_pseudo_id',
 'numerator': {'aggregation_field': 'order_id',
  'aggregation_function': 'count_distinct'},
 'numerator_conditions': [{'condition_field': 'pg_status',
   'comparison_sign': 'equal',
   'comparison_value': 'CANCELED'}],
 'denominator': {'aggregation_field': 'order_id',
  'aggregation_function': 'count_distinct'}}

In [47]:
metric_parsed = Metric(metric_params)

In [48]:
calculated_metric = CalculateMetric(metric_parsed)(df)

In [49]:
calculated_metric

,experiment_variant,user_pseudo_id,num,den,n
0,0,0006C6C0573F6696C6AD1027E8C3ED1E,0,0,1
1,0,00343880C7A39144E5E41BD5321477FD,1,1,1
2,0,00430675C2ECD7AE36BF84FF63FE2C3B,0,0,1
3,0,00954619556C2A68D6C105D8E68A812F,0,0,1
4,0,00D66BE04250EDEF1BDBAAE7FBD3A0DC,0,0,1
...,...,...,...,...,...
9595,3,FF9054B7FC8D0AC2E562A4DC337683A2,0,0,1
9596,3,FFA5E91CDB085293CCF4244F4F08024C,0,0,1
9597,3,FFA820A2B0C703068CD906AAE3DA3195,0,0,1
9598,3,FFE6E1E1DEBA56D4953E9ED7C1174BB1,0,0,1


In [21]:
df.groupby([config.VARIANT_COL, metric_parsed.level]).apply(
            lambda df: pd.Series({
                "num": self.metric.numerator_aggregation_function(df[self.metric.numerator_aggregation_field]),
                "den": self.metric.denominator_aggregation_function(df[self.metric.denominator_aggregation_field]),
                "n": pd.Series.nunique(df[self.metric.level])
            })

In [24]:
df_ = calculate_linearization(calculated_metric)

In [25]:
df_

,experiment_variant,user_pseudo_id,num,den,n,l_ratio
0,0,0006C6C0573F6696C6AD1027E8C3ED1E,0.00000,1.0,1.0,0.00000
1,0,00343880C7A39144E5E41BD5321477FD,2297.47665,1.0,1.0,2297.47665
2,0,00430675C2ECD7AE36BF84FF63FE2C3B,0.00000,1.0,1.0,0.00000
3,0,00954619556C2A68D6C105D8E68A812F,0.00000,1.0,1.0,0.00000
4,0,00D66BE04250EDEF1BDBAAE7FBD3A0DC,0.00000,1.0,1.0,0.00000
...,...,...,...,...,...,...
9595,3,FF9054B7FC8D0AC2E562A4DC337683A2,0.00000,1.0,1.0,0.00000
9596,3,FFA5E91CDB085293CCF4244F4F08024C,0.00000,1.0,1.0,0.00000
9597,3,FFA820A2B0C703068CD906AAE3DA3195,0.00000,1.0,1.0,0.00000
9598,3,FFE6E1E1DEBA56D4953E9ED7C1174BB1,0.00000,1.0,1.0,0.00000


In [38]:
stats = calculate_statistics(df_, metric_params['type'])

In [39]:
stats

In [50]:
if metric_params['estimator'] == 't_test':
    test = TTestFromStats()
elif metric_params['estimator']  == 'prop_test':
    test = ProportionZFromTest()
elif metric_params['estimator']  == 'mann_whitney':
    test = UTestFromTest()

criteria_res = test(stats)

[2023-11-25 20:42:21] - AB Pipeline - ERROR - proportions_ztest() got an unexpected keyword argument 'count_succes'


In [42]:
criteria_res.pvalue

0.3578962743580172